# Requirements:
- Define a new type called Permission that stores user permissions: read, write, and/or execute
- This type should be an enumeration, with the ability to support bitwise operations
- Separately define a new type called User, which takes a name and user_role at instantiation
- Internally, the User class sets a permissions attribute depending on the specified user_group:
  - admin: read, write, and execute
  - user: read,
  - manager: read, write
  - support: execute
- The User class also implements (or ideally inherits) read(file), write(file, content), and execute(file) methods which
are permission-checked, e.g. a User belonging to the support user_role will not be able to write, but only execute
- For ease of operation, assume that the read/write/execute functionality pertains to a python script
- Instances of User should have an informative string representation
- As an extra challenge, try to allow some polymorphism in the user_role so that it's possible to instantiate by both
string roles as well as integers, e.g. User("A", user_role=2) would imply WRITE-only permissions, whereas
User("B", user_role=6) would imply WRITE and EXEC, because 2**1 + 2**2 = 2 + 4 = 6

In [3]:
from enum import Flag, auto # to enable binary operation

In [4]:
class Permission(Flag):
    READ = auto()
    WRITE = auto()
    EXEC = auto()

In [5]:
Permission.__members__

mappingproxy({'READ': <Permission.READ: 1>,
              'WRITE': <Permission.WRITE: 2>,
              'EXEC': <Permission.EXEC: 4>})

In [6]:
# unique binary number associated with each membe
# enable bitwise operations
# READ or WRITE
# 01 or 10 -> 11 -> 3
Permission.READ | Permission.WRITE

<Permission.READ|WRITE: 3>

In [7]:
Permission(6)

<Permission.WRITE|EXEC: 6>

In [8]:
class BaseUser:

    USER_GROUP = {
        'admin': Permission.READ | Permission.WRITE | Permission.EXEC,
        'manager':Permission.READ| Permission.WRITE, 
        'user': Permission.READ,
        'support':Permission.EXEC
    }
    
    def read(self, file):
        with open(file, 'r') as f:
            for line in f:
                print(line)

    def write(self, file, content):
        with open(file, 'w') as f:
            f.write(content)

    def execute(self, file):
        with open(file, 'r') as f:
            for line in f:
                exec(line)

In [9]:
class PermissionError(Exception):
    
    def __init__(self,action):
        self.action = action
        
    def __str__(self):
        return f"User does not have {self.action.name} permission"

In [10]:
class User(BaseUser):

    def __init__(self, name, user_role):
        self.name = name
        self.user_role = user_role

    @property
    def permissions(self):
        try:
            self._permission=Permission(int(self.user_role))
        except ValueError:
            if self.user_role not in self.USER_GROUP.keys():
                raise NotImplementedError(f"{self.user_role} not defined")
            self._permission = self.USER_GROUP[self.user_role]
        return self._permission
    
    @staticmethod
    def check_permission(permission):
        def wrap(func):
            def inner(self, *args, **kwargs):
                if permission not in self.permissions:
                    raise PermissionError(permission)
                return func(self, *args, **kwargs)
            return inner
        return wrap
        
    def __repr__(self):
        return f"{type(self).__name__}(name={self.name!r}, user_role={self.user_role!r})"

    @check_permission(Permission.READ)
    def read(self, file):
        super().read(file)

    @check_permission(Permission.WRITE)
    def write(self, file, content):
        super().write(file, content)

    @check_permission(Permission.EXEC)
    def execute(self, file):
        super().execute(file)

In [11]:
u1 = User(name="Aaron", user_role="user")
u1

User(name='Aaron', user_role='user')

In [12]:
u2 = User("Andrew", "admin")
u2.permissions

<Permission.READ|WRITE|EXEC: 7>

In [13]:
u2.write(file="script.py", content="for i in range(10): print(i)")
u2.execute(file="script.py")

0
1
2
3
4
5
6
7
8
9


In [14]:
u1.read(file="script.py")

for i in range(10): print(i)


In [15]:
try:
    u1.execute(file="script.py")
except PermissionError as err:
    print(err)

User does not have EXEC permission


In [16]:
u3 = User("Joseph", 6)
u3.permissions

<Permission.WRITE|EXEC: 6>

In [17]:
u3.execute(file="script.py")

0
1
2
3
4
5
6
7
8
9
